世界の人口GDP情報は引っこ抜きたい。
https://www.stat.go.jp/data/sekai/qa-1.html#qapopdb

In [16]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install japanize_matplotlib
!pip install optuna
import japanize_matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import LabelEncoder
import datetime as dt
import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import itertools

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

In [18]:
STUDY_EDA = False
DEBUG = False

input_path = '/content/drive/MyDrive/input/signate/sony_competition/'
output_path = '/content/drive/MyDrive/output/signate/sony_competition/'
interm_path = '/content/drive/MyDrive/intermediate/signate/sony_competition/'
now = dt.datetime.today()
month = str(now.month)
day = str(now.day)
comment = 'sony_competition'

save_nm = f'submission_{comment}_{month}_{day}.csv'
print(save_nm)

submission_sony_competition_4_16.csv


In [19]:
train_df = pd.read_csv(input_path + 'train.csv')
test_df = pd.read_csv(input_path + 'test.csv')
population_df = pd.read_excel(input_path + 'WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx', sheet_name = 'ESTIMATES')

In [20]:
population_df.columns.tolist()

['Index',
 'Variant',
 'Region, subregion, country or area *',
 'Notes',
 'Country code',
 'Type',
 'Parent code',
 '1950',
 '1951',
 '1952',
 '1953',
 '1954',
 '1955',
 '1956',
 '1957',
 '1958',
 '1959',
 '1960',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020']

In [21]:
population_df = population_df[[
    'Region, subregion, country or area *',
    '1950',
    '1951',
    '1952',
    '1953',
    '1954',
    '1955',
    '1956',
    '1957',
    '1958',
    '1959',
    '1960',
    '1961',
    '1962',
    '1963',
    '1964',
    '1965',
    '1966',
    '1967',
    '1968',
    '1969',
    '1970',
    '1971',
    '1972',
    '1973',
    '1974',
    '1975',
    '1976',
    '1977',
    '1978',
    '1979',
    '1980',
    '1981',
    '1982',
    '1983',
    '1984',
    '1985',
    '1986',
    '1987',
    '1988',
    '1989',
    '1990',
    '1991',
    '1992',
    '1993',
    '1994',
    '1995',
    '1996',
    '1997',
    '1998',
    '1999',
    '2000',
    '2001',
    '2002',
    '2003',
    '2004',
    '2005',
    '2006',
    '2007',
    '2008',
    '2009',
    '2010',
    '2011',
    '2012',
    '2013',
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
    '2019',
    '2020'
 ]].rename(columns = {'Region, subregion, country or area *':'Country'})

In [22]:
train_df.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,14.141,38.374,304.731,50,2.957,3.230,14.056,25.630,176,14.959,17.471,24.221,20.883,101,16.423,19.819,20.044,3.188,111,73.504,86.920,100.000,85.028,141,1011.703,1016.545,1018.364,10.898,52,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,8.681,11.100,10.011,83,0.349,2.349,38.600,353.549,47,0.422,5.794,9.972,9.234,80,14.911,21.140,32.584,74.298,55,10.297,11.822,13.377,2.311,155,51.918,76.736,96.533,484.188,124,1019.521,1022.992,1027.611,15.142,102,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063


# preprocessing

In [23]:
train_df['train_test'] = 'train'
test_df['train_test'] = 'test'
train_test = pd.concat([train_df, test_df], axis = 0).reset_index(drop = True)
train_test.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,train_test
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,train
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,train
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,train
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,14.141,38.374,304.731,50,2.957,3.230,14.056,25.630,176,14.959,17.471,24.221,20.883,101,16.423,19.819,20.044,3.188,111,73.504,86.920,100.000,85.028,141,1011.703,1016.545,1018.364,10.898,52,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,train
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,8.681,11.100,10.011,83,0.349,2.349,38.600,353.549,47,0.422,5.794,9.972,9.234,80,14.911,21.140,32.584,74.298,55,10.297,11.822,13.377,2.311,155,51.918,76.736,96.533,484.188,124,1019.521,1022.992,1027.611,15.142,102,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,train


In [24]:
population_df.head()

,Country,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,WORLD,2536431.018,2584034.227,2630861.69,2677609.061,2724846.754,2773019.915,2822443.254,2873306.058,2925686.68,2979576.147,3034949.715,3091843.513,3150420.761,3211000.946,3273978.272,3339583.51,3407922.631,3478770.104,3551599.436,3625680.965,3700437.042,3775760.03,3851650.588,3927780.519,4003794.178,4079480.474,4154666.827,4229505.919,4304533.599,4380506.185,4458003.466,4536996.619,4617386.526,4699569.187,4784011.517,4870921.666,4960568,5052521.998,5145425.994,5237441.434,5327231.041,5414289.383,5498919.893,5581597.598,5663150.428,5744212.93,5824891.931,5905045.647,5984794.075,6064239.033,6143493.806,6222626.531,6301773.172,6381185.141,6461159.391,6541906.956,6623517.917,6705946.643,6789088.672,6872766.988,6956823.588,7041194.168,7125827.957,7210582.041,7295290.759,7379796.967,7464021.934,7547858.9,7631091.113,7713468.205,7794798.729
1,UN development groups,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,More developed regions,814818.913,824003.512,833720.173,843787.543,854060.456,864429.512,874823.542,885207.006,895575.899,905949.166,916344.412,926751.853,937111.481,947303.398,957175.398,966624.194,975584.91,984097.406,992293.268,1000365.65,1008451.91,1016600.687,1024762.958,1032860.707,1040773.52,1048416.226,1055773.866,1062884.618,1069781.204,1076514.38,1083125.601,1089607.256,1095957.014,1102226.765,1108482.387,1114763.002,1121097.639,1127452.523,1133726.567,1139778.902,1145507.68,1150893.431,1155966.401,1160736.998,1165231.553,1169481.474,1173489.817,1177284.072,1180954.357,1184615.559,1188359.407,1192204.362,1196155.656,1200271.254,1204613.392,1209215.036,1214107.114,1219250.488,1224518.298,1229735.181,1234767.516,1239557.448,1244114.531,1248453.53,1252615.112,1256622.188,1260478.667,1264146.38,1267558.904,1270630.32,1273304.261
3,Less developed regions,1721612.105,1760030.715,1797141.517,1833821.518,1870786.298,1908590.403,1947619.712,1988099.052,2030110.781,2073626.981,2118605.303,2165091.66,2213309.28,2263697.548,2316802.874,2372959.316,2432337.721,2494672.698,2559306.168,2625315.315,2691985.132,2759159.343,2826887.63,2894919.812,2963020.658,3031064.248,3098892.961,3166621.301,3234752.395,3303991.805,3374877.865,3447389.363,3521429.512,3597342.422,3675529.13,3756158.664,3839470.361,3925069.475,4011699.427,4097662.532,4181723.361,4263395.952,4342953.492,4420860.6,4497918.875,4574731.456,4651402.114,4727761.575,4803839.718,4879623.474,4955134.399,5030422.169,5105617.516,5180913.887,5256545.999,5332691.92,5409410.803,5486696.155,5564570.374,5643031.807,5722056.072,5801636.72,5881713.426,5962128.511,6042675.647,6123174.779,6203543.267,6283712.52,6363532.209,6442837.885,6521494.468
4,Least developed countries,195427.785,199180.385,203015.198,206985.638,211133.084,215486.293,220061.424,224862.263,229882.003,235105.727,240518.397,246113.624,251900.655,257908.038,264174.076,270720.935,277573.103,284708.606,292046.003,299473.694,306916.898,314345.051,321797.948,329357.734,337142.875,345242.741,353691.447,362475.157,371577.357,380963.357,390612.544,400528.973,410740.623,421279.177,432185.306,443491.299,455193.679,467290.486,479814.027,492803.041,506275.626,520262.436,534731.004,549559.275,564581.12,579681.702,594801.271,609983.284,625342.129,641046.832,657215.854,673903.088,691061.433,708587.94,726331.171,744190.509,762132.501,780219.573,798563.425,817322.615,836614.841,856471.437,876867.234,897793.439,919222.955,941131.317,963519.718

In [25]:
population_df['Country'].unique()

array(['WORLD', 'UN development groups', 'More developed regions',
       'Less developed regions', 'Least developed countries',
       'Less developed regions, excluding least developed countries',
       'Less developed regions, excluding China',
       'Land-locked Developing Countries (LLDC)',
       'Small Island Developing States (SIDS)',
       'World Bank income groups', 'High-income countries',
       'Middle-income countries', 'Upper-middle-income countries',
       'Lower-middle-income countries', 'Low-income countries',
       'No income group available', 'Geographic regions', 'Africa',
       'Asia', 'Europe', 'Latin America and the Caribbean',
       'Northern America', 'Oceania',
       'Sustainable Development Goal (SDG) regions', 'SUB-SAHARAN AFRICA',
       'Eastern Africa', 'Burundi', 'Comoros', 'Djibouti', 'Eritrea',
       'Ethiopia', 'Kenya', 'Madagascar', 'Malawi', 'Mauritius',
       'Mayotte', 'Mozambique', 'Réunion', 'Rwanda', 'Seychelles',
       'Somalia', '

In [26]:
change_country_dict = {
    'Iran (Islamic Republic of)':'Iran',
    'Russian Federation':'Russia',
    'Republic of Korea':'South Korea',
    'China, Taiwan Province of China':'Taiwan',
    'United States of America':'United States',
    'Viet Nam':'Vietnam'
 }

population_df['Country'] = population_df['Country'].replace(change_country_dict)

In [29]:
print(train_test.shape)
tmp_df = pd.merge(
    train_test,
    population_df,
    on = 'Country',
    how = 'left'
)
print(tmp_df.shape)

(249450, 55)
(249450, 126)


In [28]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
tmp_df.dropna(how='all').dropna(how='all', axis=1)

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,co_mid,co_max,co_var,o3_cnt,o3_min,o3_mid,o3_max,o3_var,so2_cnt,so2_min,so2_mid,so2_max,so2_var,no2_cnt,no2_min,no2_mid,no2_max,no2_var,temperature_cnt,temperature_min,temperature_mid,temperature_max,temperature_var,humidity_cnt,humidity_min,humidity_mid,humidity_max,humidity_var,pressure_cnt,pressure_min,pressure_mid,pressure_max,pressure_var,ws_cnt,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,train_test,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,2.590,2.633,0.850,29,2.397,4.295,5.916,3.017,11,0.149,0.468,0.900,0.137,36,0.000,0.529,2.188,1.167,36,10.956,14.038,20.866,23.880,45,30.207,70.426,94.202,994.051,59,1004.137,1015.721,1023.398,89.588,42,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,train,8177.348,8398.224,8595.064,8782.177,8970.121,9165.7,9371.781,9587.459,9808.595,10028.799,10242.07,10445.836,10643.42,10845.179,11065.419,11313.186,11593.402,11899.644,12215.704,12518.931,12793.03,13033.13,13244.163,13431.79,13605.574,13773.29,13935.722,14092.545,14249.403,14413.002,14588.4,14777.241,14979.203,15194.644,15423.156,15663.672,15917.58,16183.159,16452.258,16713.989,16960.6,17189.235,17402.179,17603.211,17798.526,17993.083,18189.274,18387.205,18587.027,18788.187,18991.434,19194.676,19401.366,19624.163,19879.654,20178.543,20526.3,20916.339,21332.293,21750.852,22154.687,22538.002,22903.951,23254.912,23596.426,23932.499,24262.71,24584.619,24898.153,25203.2,25499.881
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,3.181,4.828,1.208,49,3.092,7.572,12.332,20.524,31,0.096,0.102,16.152,61.550,45,0.099,0.301,5.411,6.401,71,24.113,30.125,36.678,37.327,50,49.034,74.276,92.975,467.112,69,938.085,970.252,1080.000,7.320,48,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,train,8177.348,8398.224,8595.064,8782.177,8970.121,9165.7,9371.781,9587.459,9808.595,10028.799,10242.07,10445.836,10643.42,10845.179,11065.419,11313.186,11593.402,11899.644,12215.704,12518.931,12793.03,13033.13,13244.163,13431.79,13605.574,13773.29,13935.722,14092.545,14249.403,14413.002,14588.4,14777.241,14979.203,15194.644,15423.156,15663.672,15917.58,16183.159,16452.258,16713.989,16960.6,17189.235,17402.179,17603.211,17798.526,17993.083,18189.274,18387.205,18587.027,18788.187,18991.434,19194.676,19401.366,19624.163,19879.654,20178.543,20526.3,20916.339,21332.293,21750.852,22154.687,22538.002,22903.951,23254.912,23596.426,23932.499,24262.71,24584.619,24898.153,25203.2,25499.881
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,1.197,2.200,0.248,123,0.736,12.527,23.432,125.224,36,1.392,1.464,47.652,515.570,111,0.972,2.751,12.230,30.518,148,0.000,0.000,23.226,131.280,138,25.919,69.069,100.000,1292.191,153,880.000,880.000,1018.200,4655.348,125,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,train,8177.348,8398.224,8595.064,8782.177,8970.121,9165.7,9371.781,9587.459,9808.595,10028.799,10242.07,10445.836,10643.42,10845.179,11065.419,11313.186,11593.402,11899.644,12215.704,12518.931,12793.03,13033.13,13244.163,13431.79,13605.574,13773.29,13935.722,14092.545,14249.403,14413.002,14588.4,14777.241,14979.203,15194.644,15423.156,15663.672,15917.58,16183.159,16452.258,16713.989,16960.6,17189.235,17402.179,17603.211,17798.526,17993.083,18189.274,18387.205,18587.027,18788.187,18991.434,19194.676,19401.366,19624.163,19879.654,20178.543,20526.3,20916.339,21332.293,21750.852,22154.687,22538.002,22903.951,23254.912,23596.426,23932.499,24262.71,24584.619,24898.153,25203.2,25499.881
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,11.044,14.802,24.186,90,1.917,1

In [ ]:
tmp_df.isnull().sum()

id                     0
year                   0
month                  0
day                    0
Country                0
City                   0
lat                    0
lon                    0
co_cnt                 0
co_min                 0
co_mid                 0
co_max                 0
co_var                 0
o3_cnt                 0
o3_min                 0
o3_mid                 0
o3_max                 0
o3_var                 0
so2_cnt                0
so2_min                0
so2_mid                0
so2_max                0
so2_var                0
no2_cnt                0
no2_min                0
no2_mid                0
no2_max                0
no2_var                0
temperature_cnt        0
temperature_min        0
temperature_mid        0
temperature_max        0
temperature_var        0
humidity_cnt           0
humidity_min           0
humidity_mid           0
humidity_max           0
humidity_var           0
pressure_cnt           0
pressure_min           0
